In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import plotly.express as px
import xgboost

In [2]:
cd C:\Users\ComProWorld\Desktop\b7\Churn Customer Classification Project


C:\Users\ComProWorld\Desktop\b7\Churn Customer Classification Project


In [3]:
customer = pd.read_excel('CHURNDATA.xlsx')
customer.head()

CIF              CUS_DOB  AGE  CUS_Month_Income CUS_Gender  \
0  XXXXXX  Feb 13 1970 12:00AM   49           7116.64       MALE   
1  XXXXXX  Sep 20 1973 12:00AM   46        1500000.00     FEMALE   
2  XXXXXX  Jul 18 1966 12:00AM   53        5000000.00     FEMALE   
3  XXXXXX  Jan  9 2004 12:00AM   15            500.00     FEMALE   
4  XXXXXX  Mar  7 1971 12:00AM   48        9000000.00     FEMALE   

  CUS_Marital_Status CUS_Customer_Since  YEARS_WITH_US  \
0            MARRIED         1994-06-30             25   
1             SINGLE         2005-05-19             14   
2             SINGLE         2005-05-20             14   
3             SINGLE         2005-05-20             14   
4             SINGLE         2014-06-30              5   

   # total debit transactions for S1  # total debit transactions for S2  ...  \
0                                277                                265  ...   
1                                 37                                 15  ...   
2                                 31                                 14  ...   
3                                  0                                  1  ...   
4                                 15                                 33  ...   

   total credit amount for S2  total credit amount for S3  total debit amount  \
0                  1764079.61                  2378592.62          4758311.63   
1                    19500.00                    57500.00           139363.22   
2                   240311.84                    70946.86            37375.46   
3                    28089.99                    18753.92             4000.00   
4                   176000.00                    70500.00           338470.29   

   total debit transactions  total credit amount  total credit transactions  \
0                       887           5659653.33                         65   
1                        97             87000.00                         10   
2                        53           1020883.70                         27   
3                         1             76946.96                         12   
4                        60            442100.00                         21   

   total transactions  CUS_Target   TAR_Desc  Status  
0                 952        2231  EXECUTIVE  ACTIVE  
1                 107        2223        LOW  ACTIVE  
2                  80        2222      MIDLE  ACTIVE  
3                  13        2223        LOW  ACTIVE  
4                  81        2223        LOW  ACTIVE  

[5 rows x 28 columns]

In [4]:
customer.isnull().sum()

CIF                                    0
CUS_DOB                                0
AGE                                    0
CUS_Month_Income                      11
CUS_Gender                             2
CUS_Marital_Status                     0
CUS_Customer_Since                     0
YEARS_WITH_US                          0
# total debit transactions for S1      0
# total debit transactions for S2      0
# total debit transactions for S3      0
total debit amount for S1              0
total debit amount for S2              0
total debit amount for S3              0
# total credit transactions for S1     0
# total credit transactions for S2     0
# total credit transactions for S3     0
total credit amount for S1             0
total credit amount for S2             0
total credit amount for S3             0
total debit amount                     0
total debit transactions               0
total credit amount                    0
total credit transactions              0
total transactio

In [5]:
customer = customer.drop(customer.index[[85,99,184,316,368,400,427,857,938,996,1047]])
customer['CUS_Month_Income'] = pd.to_numeric(customer['CUS_Month_Income'])
customer['CUS_Gender'] = customer['CUS_Gender'].fillna('FEMALE')

In [6]:
customer['CUS_Month_Income'] = customer['CUS_Month_Income'].astype(int)
del customer['CIF']
del customer["CUS_DOB"]
del customer['CUS_Customer_Since']

In [7]:
customer.columns

Index(['AGE', 'CUS_Month_Income', 'CUS_Gender', 'CUS_Marital_Status',
       'YEARS_WITH_US', '# total debit transactions for S1',
       '# total debit transactions for S2',
       '# total debit transactions for S3', 'total debit amount for S1',
       'total debit amount for S2', 'total debit amount for S3',
       '# total credit transactions for S1',
       '# total credit transactions for S2',
       '# total credit transactions for S3', 'total credit amount for S1',
       'total credit amount for S2', 'total credit amount for S3',
       'total debit amount', 'total debit transactions', 'total credit amount',
       'total credit transactions', 'total transactions', 'CUS_Target',
       'TAR_Desc', 'Status'],
      dtype='object')

In [8]:
churn = customer.copy()

In [9]:
from sklearn.preprocessing import LabelEncoder
status = LabelEncoder()
churn['Status'] = status.fit_transform(churn['Status'])


In [10]:
X = churn[['CUS_Month_Income',
                  '# total debit transactions for S3', 'total debit amount for S3',
                  '# total credit transactions for S3', 'total credit amount for S3',
                  '# total debit transactions for S2', 'total debit amount for S2',
                  '# total credit transactions for S2', 'total credit amount for S2',
                  'total debit amount','total credit transactions','total transactions']]

In [11]:
y = churn['Status']

In [12]:
X.head()

CUS_Month_Income  # total debit transactions for S3  \
0              7116                                345   
1           1500000                                 45   
2           5000000                                  8   
3               500                                  0   
4           9000000                                 12   

   total debit amount for S3  # total credit transactions for S3  \
0                 2068641.91                                  31   
1                   83856.67                                   4   
2                    4348.95                                   5   
3                       0.00                                   3   
4                   46379.40                                   6   

   total credit amount for S3  # total debit transactions for S2  \
0                  2378592.62                                265   
1                    57500.00                                 15   
2                    70946.86                                 14   
3                    18753.92                                  1   
4                    70500.00                                 33   

   total debit amount for S2  # total credit transactions for S2  \
0                 1230543.08                                  24   
1                   20134.00                                   4   
2                    4777.66                                   8   
3                    4000.00                                   4   
4                  223007.51                                   7   

   total credit amount for S2  total debit amount  total credit transactions  \
0                  1764079.61          4758311.63                         65   
1                    19500.00           139363.22                         10   
2                   240311.84            37375.46                         27   
3                    28089.99             4000.00                         12   
4                   176000.00           338470.29                         21   

   total transactions  
0                 952  
1                 107  
2                  80  
3                  13  
4                  81

In [13]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Status, dtype: int32

In [14]:
from imblearn.combine import SMOTETomek
smk = SMOTETomek(random_state=1234)
X_res, y_res = smk.fit_resample(X,y)
X_res.shape, y_res.shape

((1940, 12), (1940,))

In [15]:
from collections import Counter
print('Original dataset shape {}', format(Counter(y)))
print('resampled dataset shape {}', format(Counter(y_res)))

Original dataset shape {} Counter({0: 1015, 1: 223})
resampled dataset shape {} Counter({0: 970, 1: 970})


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res,train_size=0.8)

In [17]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1552, 12)
(1552,)
(388, 12)
(388,)


In [18]:
from sklearn.ensemble import GradientBoostingClassifier
model_gb = GradientBoostingClassifier(learning_rate=0.02, n_estimators=225, max_depth=7,min_samples_leaf=2,warm_start=True,
                                      min_samples_split=3,max_features='sqrt',random_state=1234, loss='exponential')

In [19]:
model_gb.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.02, loss='exponential', max_depth=7,
                           max_features='sqrt', min_samples_leaf=2,
                           min_samples_split=3, n_estimators=225,
                           random_state=1234, warm_start=True)

In [20]:
y_pred = model_gb.predict(X_test)
y_pred

array([1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,

In [21]:
model_gb.score(X_train,y_train)

0.9864690721649485

In [22]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.904639175257732

In [23]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,y_pred))

[[177  28]
 [  9 174]]


In [24]:
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.86      0.91       205
           1       0.86      0.95      0.90       183

    accuracy                           0.90       388
   macro avg       0.91      0.91      0.90       388
weighted avg       0.91      0.90      0.90       388



In [25]:
import pickle

savefile = open("C:/Users/ComProWorld/Desktop/b7/Churn Customer Classification Project/Churn Project Deployment/Model_Gb.pickle", "wb")
pickle.dump(model_gb, savefile)
savefile.close()
readfile = open("Model_Gb.pickle", "rb")
model = pickle.load(readfile)
readfile.close()

print(model.predict([[45000,220,40000,120,50000,330,60000,120,80000,150000,620,800]]))

[0]
